# Loading and pre-processing dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import random,time,csv
import numpy as np
import math,copy,os
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import sklearn.metrics as metrics
import os
import tempfile
import pandas as pd
import six.moves.urllib as urllib
import pprint
! pip install fairlearn
import random
import pandas as pd
import random,time,csv
import numpy as np
import math,copy,os
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import sklearn.metrics as metrics
import os
import tempfile
import pandas as pd
import six.moves.urllib as urllib
import pprint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from pandas import read_csv
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from google.colab import files
from fairlearn.reductions import BoundedGroupLoss, ZeroOneLoss
from sklearn.metrics import mean_absolute_error
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from fairlearn.metrics import MetricFrame, selection_rate
from sklearn.preprocessing import MinMaxScaler
from google.protobuf import text_format

import tensorflow as tf
tf.compat.v1.enable_v2_behavior()
import sys
sys.path.insert(1, '/content/drive/Shareddrives/622 - Privacy/Project Codebase/Fair-SMOTE-master')
from Measure import measure_final_score,calculate_recall,calculate_far,calculate_precision,calculate_accuracy
from Generate_Samples import generate_samples

     |████████████████████████████████| 177 kB 29.0 MB/s 


In [ ]:
# Loading and keeping only the required columns

_DATA_PATH = 'https://storage.googleapis.com/lawschool_dataset/bar_pass_prediction.csv'

data = urllib.request.urlopen(_DATA_PATH)

_LSAT_DF = pd.read_csv(data)


_COLUMN_NAMES = [
  'dnn_bar_pass_prediction',
  'gender',
  'lsat',
  'pass_bar',
  'race1',
  'ugpa',
]

_LSAT_DF.dropna()
_LSAT_DF['gender'] = _LSAT_DF['gender'].astype(str)
_LSAT_DF['race1'] = _LSAT_DF['race1'].astype(str)
df = _LSAT_DF[_COLUMN_NAMES]

df.head(3)

,dnn_bar_pass_prediction,gender,lsat,pass_bar,race1,ugpa
0,0.979804,female,44.0,1.0,white,3.5
1,0.979804,female,29.0,1.0,white,3.5
2,0.979804,male,36.0,1.0,white,3.5


In [ ]:
df.describe()
# taking 50% i.e. 3.2 as the threshold 

,dnn_bar_pass_prediction,lsat,pass_bar,ugpa
count,22407.000000,22407.000000,22407.000000,22407.000000
mean,0.980682,36.767907,0.947829,3.215451
std,0.022594,5.463248,0.222377,0.404073
min,0.330740,11.000000,0.000000,1.500000
25%,0.971071,33.000000,1.000000,3.000000
50%,0.983144,37.000000,1.000000,3.200000
75%,0.996526,41.000000,1.000000,3.500000
max,0.999698,48.000000,1.000000,3.900000


In [ ]:
# creating a new binary column based on the second quartile of ugpa column
# dropping null values and one hot encoding race1 columns 
# PROTECTED ATTRIBUTE IN THIS DATASET IS RACE 

second_quartile = 3.2

threshold = second_quartile# taking first quartile of the ugpa as the threshold to create binary column
df['ugpa_above_threshold'] = df.apply(lambda row: 1 if row['ugpa'] > threshold else 0, axis=1)

df = df.dropna()
df['gender'] = np.where(df['gender'] == 'male', 1, 0)

df = df[df['race1'] != 'other']
df = df[df['race1'] != 'nan']
race_one_hot = pd.get_dummies(df.race1)
_ = df.drop('race1', axis = 1)
data = pd.concat([_,race_one_hot],axis = 1)

scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data),columns = data.columns)

data.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,dnn_bar_pass_prediction,gender,lsat,pass_bar,ugpa,ugpa_above_threshold,asian,black,hisp,white
0,0.970262,0.0,0.891892,1.0,0.833333,1.0,0.0,0.0,0.0,1.0
1,0.970262,0.0,0.486486,1.0,0.833333,1.0,0.0,0.0,0.0,1.0
2,0.970262,1.0,0.675676,1.0,0.833333,1.0,0.0,0.0,0.0,1.0


In [ ]:
data_original = data # saving for linear regression on original dataset

In [ ]:
data.describe()

,dnn_bar_pass_prediction,gender,lsat,pass_bar,ugpa,ugpa_above_threshold,asian,black,hisp,white
count,21983.000000,21983.000000,21983.000000,21983.000000,21983.000000,21983.000000,21983.000000,21983.000000,21983.000000,21983.000000
mean,0.971683,0.560751,0.697155,0.948824,0.715371,0.499522,0.040804,0.061093,0.046718,0.851385
std,0.033513,0.496307,0.147135,0.220361,0.168155,0.500011,0.197841,0.239506,0.211039,0.355716
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.957207,0.000000,0.594595,1.000000,0.625000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.975254,1.000000,0.702703,1.000000,0.708333,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.995258,1.000000,0.810811,1.000000,0.833333,1.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
threshold = 0.708 # saving normalised second quartile for Statistical Parity


# Oversampling and removing biased labels using Fair-SMOTE

## Oversampling 

In [ ]:
# Creating subgroups according to race and values of ugpa_above_average column
# checking instance count of different subgroups
black_ab_av = len(data[(data['ugpa_above_threshold'] == 1) & (data['black'] == 1)])
black_bl_av= len(data[(data['ugpa_above_threshold'] == 0) & (data['black'] == 1)])
white_ab_av= len(data[(data['ugpa_above_threshold'] == 1) & (data['white'] == 1)])
white_bl_av= len(data[(data['ugpa_above_threshold'] == 0) & (data['white'] == 1)])
hisp_ab_av= len(data[(data['ugpa_above_threshold'] == 1) & (data['hisp'] == 1)])
hisp_bl_av= len(data[(data['ugpa_above_threshold'] == 0) & (data['hisp'] == 1)])
asian_ab_av= len(data[(data['ugpa_above_threshold'] == 1) & (data['asian'] == 1)])
asian_bl_av= len(data[(data['ugpa_above_threshold'] == 0) & (data['asian'] == 1)])

print('Instance count of different columns is --',black_ab_av,black_bl_av,white_ab_av,white_bl_av,hisp_ab_av,hisp_bl_av,asian_ab_av,asian_bl_av)


Instance count of different columns is -- 298 1045 9861 8855 362 665 460 437


In [ ]:
maximum = max(black_ab_av,black_bl_av,white_ab_av,white_bl_av,hisp_ab_av,hisp_bl_av,asian_ab_av,asian_bl_av)
# white students with ugpa > average are more 
# therefore, increasing size of other subgroups
black_ab_av_to_be_incresed = maximum - black_ab_av
black_bl_av_to_be_incresed = maximum - black_bl_av 
white_bl_av_to_be_incresed = maximum - white_bl_av 
hisp_ab_av_to_be_incresed = maximum - hisp_ab_av
hisp_bl_av_to_be_incresed = maximum - hisp_bl_av 
asian_ab_av_to_be_incresed = maximum - asian_ab_av 
asian_bl_av_to_be_incresed = maximum - asian_bl_av 

df_black_ab_av = (data[(data['ugpa_above_threshold'] == 1) & (data['black'] == 1)])
df_black_bl_av= (data[(data['ugpa_above_threshold'] == 0) & (data['black'] == 1)])
df_white_bl_av= (data[(data['ugpa_above_threshold'] == 0) & (data['white'] == 1)])
df_hisp_ab_av= (data[(data['ugpa_above_threshold'] == 1) & (data['hisp'] == 1)])
df_hisp_bl_av= (data[(data['ugpa_above_threshold'] == 0) & (data['hisp'] == 1)])
df_asian_ab_av= (data[(data['ugpa_above_threshold'] == 1) & (data['asian'] == 1)])
df_asian_bl_av= (data[(data['ugpa_above_threshold'] == 0) & (data['asian'] == 1)])

In [ ]:
# generating samples and making instance counts equal to the count of largest subgroup 
# using Generate samples function given by FairSMOTE authors. It uses KNN
df_black_ab_av = generate_samples(black_ab_av_to_be_incresed,df_black_ab_av,'law').rename(columns={0:"dnn_bar_pass_prediction",1:"gender",2:"lsat",3:"pass_bar",4:'ugpa',5:"ugpa_above_threshold",6:"asian",7:"black",8:"hisp",9:"white"})
df_black_bl_av= generate_samples(black_bl_av_to_be_incresed,df_black_bl_av,'law').rename(columns={0:"dnn_bar_pass_prediction",1:"gender",2:"lsat",3:"pass_bar",4:'ugpa',5:"ugpa_above_threshold",6:"asian",7:"black",8:"hisp",9:"white"})
df_white_bl_av= generate_samples(white_bl_av_to_be_incresed,df_white_bl_av,'law').rename(columns={0:"dnn_bar_pass_prediction",1:"gender",2:"lsat",3:"pass_bar",4:'ugpa',5:"ugpa_above_threshold",6:"asian",7:"black",8:"hisp",9:"white"})
df_hisp_ab_av= generate_samples(hisp_ab_av_to_be_incresed,df_hisp_ab_av,'law').rename(columns={0:"dnn_bar_pass_prediction",1:"gender",2:"lsat",3:"pass_bar",4:'ugpa',5:"ugpa_above_threshold",6:"asian",7:"black",8:"hisp",9:"white"})
df_hisp_bl_av= generate_samples(hisp_bl_av_to_be_incresed,df_hisp_bl_av,'law').rename(columns={0:"dnn_bar_pass_prediction",1:"gender",2:"lsat",3:"pass_bar",4:'ugpa',5:"ugpa_above_threshold",6:"asian",7:"black",8:"hisp",9:"white"})
df_asian_ab_av= generate_samples(asian_ab_av_to_be_incresed,df_asian_ab_av,'law').rename(columns={0:"dnn_bar_pass_prediction",1:"gender",2:"lsat",3:"pass_bar",4:'ugpa',5:"ugpa_above_threshold",6:"asian",7:"black",8:"hisp",9:"white"})
df_asian_bl_av= generate_samples(asian_bl_av_to_be_incresed,df_asian_bl_av,'law').rename(columns={0:"dnn_bar_pass_prediction",1:"gender",2:"lsat",3:"pass_bar",4:'ugpa',5:"ugpa_above_threshold",6:"asian",7:"black",8:"hisp",9:"white"})

In [ ]:
df_white_ab_av= (data[(data['ugpa_above_threshold'] == 1) & (data['white'] == 1)])
# combining all the oversampled aubgroups
data_oversampled_main = df_white_ab_av.append([df_black_ab_av,df_black_bl_av,df_white_bl_av,df_hisp_ab_av,df_hisp_bl_av,df_asian_ab_av,df_asian_bl_av])
data_oversampled = data_oversampled_main
print('The oversampled data has shape --',data_oversampled.shape)

The oversampled data has shape -- (78888, 10)


## Removal of biased data points using situation testing


In [ ]:
data_oversampled['new_index'] = range(0,78888) # fair SMOTE implementation return repetitive indexes according to the subgroup
_data_oversampled = data_oversampled.set_index('new_index')

X_train, y_train = _data_oversampled.loc[:, _data_oversampled.columns != 'ugpa_above_threshold'], _data_oversampled['ugpa_above_threshold']

clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf.fit(X_train,y_train)
removal_list = []

for index,row in X_train.iterrows():
    row_ = [row.values[0:len(row.values)]]    
    y_normal = clf.predict(row_)

    #switching for asian

    if row_[0][5] == 1:
      change_to_1 = random.choice([6,7,8])
      row_[0][5] = 0
      row_[0][change_to_1] = 1   
      y_reverse = clf.predict(row_)
      if y_normal[0] != y_reverse[0]:
          removal_list.append(index)

    #switching for black

    if row_[0][6] == 1:
      change_to_1 = random.choice([5,7,8])
      row_[0][6] = 0
      row_[0][change_to_1] = 1   
      y_reverse = clf.predict(row_)
      if y_normal[0] != y_reverse[0]:
          removal_list.append(index)

    #switching for hisp

    if row_[0][7] == 1:
      change_to_1 = random.choice([5,6,8])
      row_[0][7] = 0
      row_[0][change_to_1] = 1   
      y_reverse = clf.predict(row_)
      if y_normal[0] != y_reverse[0]:
          removal_list.append(index)

    #switching for white

    if row_[0][8] == 1:
      change_to_1 = random.choice([5,6,7])
      row_[0][8] = 0
      row_[0][change_to_1] = 1   
      y_reverse = clf.predict(row_)
      if y_normal[0] != y_reverse[0]:
          removal_list.append(index)


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but Log

In [ ]:
removal_list = list(set(removal_list))

print(f'A total of {len(removal_list)} instances are biased and will be removed')
_df = data_oversampled
_remov = removal_list

A total of 330 instances are biased and will be removed


In [ ]:
# removing biased datapoints 
final = _data_oversampled.drop(_data_oversampled.index[list(set(removal_list))])
print('Size of final dataset is ',len(final))

Size of final dataset is  78558


In [ ]:
# uncomment to save dataset

# final.to_csv('fairSMOTE.csv')

# Evaluation 

## Linear Regression on balanced dataset

In [ ]:
# training linear regression model to evaluate 
data_oversampled_onlyugpa = final.drop('ugpa_above_threshold',axis = 1)
df = data_oversampled_onlyugpa
X = df.iloc[:,df.columns != 'ugpa']

Y = df['ugpa']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

model = linear_model.LinearRegression()
model.fit(X_train, Y_train)
y_pred_w_prot_attr = model.predict(X_test)

rmsd = np.sqrt(mean_squared_error(Y_test, y_pred_w_prot_attr))      
r2_value = r2_score(Y_test, y_pred_w_prot_attr)                     
                   
print('Linear regression with protected attribute')

print("Intercept: ", model.intercept_)
print("Root Mean Square Error: ", rmsd)
print("R^2 Value: ", r2_value)

Linear regression with protected attribute
Intercept:  1095703839021.861
Root Mean Square Error:  0.1321537800509828
R^2 Value:  0.4335072493338923


In [ ]:
# Linear regression without protected attribute 
_df = final.drop(['asian','black','hisp','white','ugpa_above_threshold'], axis = 1)
X = _df.iloc[:,_df.columns != 'ugpa']
Y = _df['ugpa']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

model = linear_model.LinearRegression()
model.fit(X_train, Y_train)
y_pred_wo_prot_attr = model.predict(X_test)

rmsd = np.sqrt(mean_squared_error(Y_test, y_pred_wo_prot_attr))      
r2_value = r2_score(Y_test, y_pred_wo_prot_attr)  
                   
print('Linear regression without protected attribute')
print("Intercept: ", model.intercept_)
print("Root Mean Square Error: ", rmsd)
print("R^2 Value: ", r2_value)

Linear regression without protected attribute
Intercept:  -1.988892775335207
Root Mean Square Error:  0.13229997480195116
R^2 Value:  0.4322531940156805


In [ ]:
# Statistical Parity

averageCounter = 0
for entry in y_pred_w_prot_attr:
  if entry > threshold : averageCounter+=1
prob_w_prot_attr = averageCounter/len(y_pred_w_prot_attr)


averageCounter = 0
for entry in y_pred_wo_prot_attr:
  if entry > threshold : averageCounter+=1
prob_wo_prot_attr = averageCounter/len(y_pred_wo_prot_attr)

print('Statistical Parity is ', abs(prob_w_prot_attr-prob_wo_prot_attr))

Statistical Parity is  0.001081975560081494


In [ ]:
# Bounded Group Loss

def apply_complex_function(x): 
  if x['black'] == 1:
    return 0
  if x['asian'] == 1:
    return 2
  if x['white'] == 1:
    return 1
  if x['hisp'] == 1:
    return 3

df = final 
df['race'] = df.apply(apply_complex_function, axis=1) 
  
  
df = df.drop(['asian','black','hisp','white'], axis = 1)
X = df.iloc[:,df.columns != 'ugpa']
Y = df['ugpa']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)


bgl = BoundedGroupLoss(ZeroOneLoss(), upper_bound=0.1)
sensitive_features = np.array(list(X_test['race'].values))
mae_frame = MetricFrame(mean_absolute_error,
                        Y_test, y_pred_w_prot_attr,
                        sensitive_features=pd.Series(sensitive_features, name="SF 0"))
print(mae_frame.overall)

0.1075939393415794


/usr/local/lib/python3.7/dist-packages/fairlearn/metrics/_metric_frame.py:67: FutureWarning: You have provided 'metrics', 'y_true', 'y_pred' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  FutureWarning)
